# Setup setting and seed

In [1]:
from multilevel_sabre import MultiLevelSabre
from pathlib import Path
from typing import Dict, List, Iterable
from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.passes import SabreLayout
from qiskit.transpiler import Layout
from qiskit.transpiler import CouplingMap, PassManager
from IPython.display import display
import json
import time
import pandas as pd


seed = 42

# Setup circuits

In [ ]:
def gather_qasm_files(paths: Iterable[str]) -> List[Path]:
    """Expand a list of files/dirs into a sorted list of .qasm files."""
    result: List[Path] = []
    for p in paths:
        path = Path(p)
        if path.is_dir():
            result.extend(sorted(path.glob("*.qasm")))
        elif path.is_file() and path.suffix.lower() == ".qasm":
            result.append(path)
        else:
            print(f"Skipping {p}: not a .qasm file or directory.")
    return sorted(set(result))

filepath = "circuits/"
qasm_files = gather_qasm_files([filepath])
print(f"Found {len(qasm_files)} .qasm files in {filepath}")

Found 7 .qasm files in circuits_iso/


In [3]:
circuits_dict = {}
for q in qasm_files:
    circuit = QuantumCircuit.from_qasm_file(q)
    print(f"Visualizing {q.name} with {circuit.num_qubits} qubits and {circuit.size()} gates")
    #display(circuit.draw('mpl', idle_wires=False, fold=-1))
    circuits_dict[q.name] = circuit


Visualizing cat_n65.qasm with 65 qubits and 130 gates
Visualizing dnn_n51.qasm with 51 qubits and 274 gates
Visualizing ghz_n78.qasm with 78 qubits and 156 gates
Visualizing ising_n98.qasm with 98 qubits and 1170 gates
Visualizing knn_67.qasm with 67 qubits and 102 gates
Visualizing multiplier_n45.qasm with 45 qubits and 698 gates
Visualizing swap_test_n83.qasm with 83 qubits and 126 gates


In [4]:
circuits_dict

{'cat_n65.qasm': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x11d2f0bd0>,
 'dnn_n51.qasm': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x11968f490>,
 'ghz_n78.qasm': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x11ff10390>,
 'ising_n98.qasm': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x11ff20750>,
 'knn_67.qasm': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x119694a10>,
 'multiplier_n45.qasm': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x11ff20550>,
 'swap_test_n83.qasm': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x11af1f4d0>}

# Setup coupling map

In [5]:
from qiskit_ibm_runtime.fake_provider.backends.sherbrooke import FakeSherbrooke
fake_backend = FakeSherbrooke()
coupling_map = fake_backend.coupling_map   
coupling_map.make_symmetric()
print(f"Number of qubits: {coupling_map.size()}")
# coupling_map.draw()


Number of qubits: 127


# Setup Passmangers

In [6]:
pm_preset = generate_preset_pass_manager(coupling_map=coupling_map, optimization_level=2, seed_transpiler=seed)


In [7]:
pm_multi_sabre = PassManager([
    MultiLevelSabre(
        coupling_graph=coupling_map,
        cycles=10,
        random_seed=seed,
        coarsest_solving_trials=50,
        num_interpolation=10,
        use_initial_embedding=True,
        verbose=0,
    )
])

In [8]:
number_of_trial = 1
pm_sabre = PassManager([
    SabreLayout(
        coupling_map = coupling_map, 
        seed = seed, 
        layout_trials=number_of_trial, 
        skip_routing=False)
])

# Transpile circuit

In [9]:
def count_swaps(circuit):
    return sum(1 for instruction in circuit.data if instruction.operation.name == 'swap')

In [10]:
qc_name = "cat_n65.qasm"
qc = circuits_dict[qc_name]

def transpile_circuit(qc, pm, pm_name, qc_name):
    t0 = time.time()
    qc_tr = pm.run(qc)

    transpile_time = time.time() - t0
    num_swaps = count_swaps(qc_tr)
    depth = qc_tr.depth()
    depth_2q = qc_tr.depth(lambda x: x.operation.num_qubits == 2)
    num_gates = qc_tr.size()
    ops = json.dumps(qc_tr.count_ops(), default=int)

    tr_info = {
        "name": qc_name,
        "num_qubits": qc.num_qubits,
        "num_gates": num_gates,
        "depth": depth,
        "depth_2q": depth_2q,
        "num_swaps": num_swaps,
        "transpile_time": transpile_time,
        "ops": ops,
        "passmanager": pm_name
    }

    return qc_tr, tr_info

In [11]:
# Preset pass manager
qc_tr_preset, info_preset = transpile_circuit(qc, pm_preset, "preset", qc_name)
print(f"Preset PassManager: {info_preset}")
print(f"Number of SWAPs (preset): {count_swaps(qc_tr_preset)}")
print(f"2Q Depth (preset): {qc_tr_preset.depth(lambda x: x.operation.num_qubits == 2)}\n")

# Sabre pass manager
qc_tr_sabre, info_sabre = transpile_circuit(qc, pm_sabre, "sabre", qc_name)
print(f"Sabre PassManager: {info_sabre}")
print(f"Number of SWAPs (sabre): {count_swaps(qc_tr_sabre)}")
print(f"2Q Depth (sabre): {qc_tr_sabre.depth(lambda x: x.operation.num_qubits == 2)}\n")

# Multi-level Sabre pass manager
qc_tr_multi_sabre, info_multi_sabre = transpile_circuit(qc, pm_multi_sabre, "multi_sabre", qc_name)
print(f"\nMulti-level Sabre PassManager: {info_multi_sabre}")
print(f"Number of SWAPs (multi_sabre): {count_swaps(qc_tr_multi_sabre)}")
print(f"2Q Depth (multi_sabre): {qc_tr_multi_sabre.depth(lambda x: x.operation.num_qubits == 2)}\n")


Preset PassManager: {'name': 'cat_n65.qasm', 'num_qubits': 65, 'num_gates': 144, 'depth': 80, 'depth_2q': 78, 'num_swaps': 14, 'transpile_time': 0.029728174209594727, 'ops': '{"measure": 65, "cx": 64, "swap": 14, "h": 1, "barrier": 1}', 'passmanager': 'preset'}
Number of SWAPs (preset): 14
2Q Depth (preset): 78

Sabre PassManager: {'name': 'cat_n65.qasm', 'num_qubits': 65, 'num_gates': 144, 'depth': 80, 'depth_2q': 78, 'num_swaps': 14, 'transpile_time': 0.006617069244384766, 'ops': '{"measure": 65, "cx": 64, "swap": 14, "h": 1, "barrier": 1}', 'passmanager': 'sabre'}
Number of SWAPs (sabre): 14
2Q Depth (sabre): 78

{0: 95, 1: 94, 2: 90, 3: 75, 4: 76, 5: 77, 6: 78, 7: 79, 8: 80, 9: 81, 10: 72, 11: 62, 12: 63, 13: 64, 14: 54, 15: 45, 16: 46, 17: 47, 18: 48, 19: 49, 20: 50, 21: 51, 22: 36, 23: 32, 24: 31, 25: 30, 26: 29, 27: 28, 28: 27, 29: 26, 30: 25, 31: 24, 32: 34, 33: 43, 34: 42, 35: 41, 36: 53, 37: 60, 38: 59, 39: 58, 40: 57, 41: 56, 42: 52, 43: 37, 44: 38, 45: 39, 46: 33, 47: 20, 4

# Run all circuits

In [12]:
def run_experiment_for_pm(pm, pm_name, circuits_dict):
    results = []
    qc_tr_list = []
    for qc_name, qc in circuits_dict.items():
        print(f"Transpiling {qc_name} with {pm_name}...")
        qc_tr, info = transpile_circuit(qc, pm, pm_name, qc_name)
        qc_tr_list.append(qc_tr)
        results.append(info)
    return results, qc_tr_list

In [13]:
# pm preset
results_preset, qc_tr_list_preset = run_experiment_for_pm(pm_preset, "preset", circuits_dict)
# display results as a table
df_preset = pd.DataFrame(results_preset)
display(df_preset)

# pm sabre
results_sabre, qc_tr_list_sabre = run_experiment_for_pm(pm_sabre, "sabre", circuits_dict)
# display results as a table
df_sabre = pd.DataFrame(results_sabre)
display(df_sabre)

# pm multi_sabre
results_multi_sabre, qc_tr_list_multi_sabre = run_experiment_for_pm(pm_multi_sabre, "multi_sabre", circuits_dict)
# display results as a table
df_multi_sabre = pd.DataFrame(results_multi_sabre)
display(df_multi_sabre)

Transpiling cat_n65.qasm with preset...
Transpiling dnn_n51.qasm with preset...
Transpiling ghz_n78.qasm with preset...
Transpiling ising_n98.qasm with preset...
Transpiling knn_67.qasm with preset...
Transpiling multiplier_n45.qasm with preset...
Transpiling swap_test_n83.qasm with preset...


,name,num_qubits,num_gates,depth,depth_2q,num_swaps,transpile_time,ops,passmanager
0,cat_n65.qasm,65,144,80,78,14,0.026400,"{""measure"": 65, ""cx"": 64, ""swap"": 14, ""h"": 1, ...",preset
1,dnn_n51.qasm,51,230,70,65,6,0.050525,"{""measure"": 51, ""u3"": 50, ""cswap"": 25, ""rzz"": ...",preset
2,ghz_n78.qasm,78,172,95,93,16,0.025341,"{""measure"": 78, ""cx"": 77, ""swap"": 16, ""h"": 1, ...",preset
3,ising_n98.qasm,98,552,18,10,17,0.050172,"{""cx"": 194, ""rz"": 145, ""u2"": 98, ""measure"": 98...",preset
4,knn_67.qasm,67,102,36,0,0,0.010676,"{""ry"": 66, ""cswap"": 33, ""h"": 2, ""measure"": 1}",preset
5,multiplier_n45.qasm,45,698,462,234,0,0.024032,"{""ccx"": 378, ""cx"": 306, ""measure"": 9, ""x"": 5}",preset
6,swap_test_n83.qasm,83,126,44,0,0,0.010623,"{""rx"": 82, ""cswap"": 41, ""h"": 2, ""measure"": 1}",preset


Transpiling cat_n65.qasm with sabre...
Transpiling dnn_n51.qasm with sabre...
Transpiling ghz_n78.qasm with sabre...
Transpiling ising_n98.qasm with sabre...
Transpiling knn_67.qasm with sabre...
Transpiling multiplier_n45.qasm with sabre...
Transpiling swap_test_n83.qasm with sabre...


,name,num_qubits,num_gates,depth,depth_2q,num_swaps,transpile_time,ops,passmanager
0,cat_n65.qasm,65,144,80,78,14,0.005376,"{""measure"": 65, ""cx"": 64, ""swap"": 14, ""h"": 1, ...",sabre
1,dnn_n51.qasm,51,280,71,65,6,0.006216,"{""measure"": 51, ""ry"": 50, ""rz"": 50, ""cswap"": 2...",sabre
2,ghz_n78.qasm,78,172,95,93,16,0.005726,"{""measure"": 78, ""cx"": 77, ""swap"": 16, ""h"": 1, ...",sabre
3,ising_n98.qasm,98,1187,29,10,17,0.012764,"{""rz"": 584, ""h"": 294, ""cx"": 194, ""measure"": 98...",sabre
4,knn_67.qasm,67,102,36,0,0,0.003397,"{""ry"": 66, ""cswap"": 33, ""h"": 2, ""measure"": 1}",sabre
5,multiplier_n45.qasm,45,698,462,234,0,0.008815,"{""ccx"": 378, ""cx"": 306, ""measure"": 9, ""x"": 5}",sabre
6,swap_test_n83.qasm,83,126,44,0,0,0.003435,"{""rx"": 82, ""cswap"": 41, ""h"": 2, ""measure"": 1}",sabre


Transpiling cat_n65.qasm with multi_sabre...
{0: 95, 1: 94, 2: 90, 3: 75, 4: 76, 5: 77, 6: 78, 7: 79, 8: 80, 9: 81, 10: 72, 11: 62, 12: 63, 13: 64, 14: 54, 15: 45, 16: 46, 17: 47, 18: 48, 19: 49, 20: 50, 21: 51, 22: 36, 23: 32, 24: 31, 25: 30, 26: 29, 27: 28, 28: 27, 29: 26, 30: 25, 31: 24, 32: 34, 33: 43, 34: 42, 35: 41, 36: 53, 37: 60, 38: 59, 39: 58, 40: 57, 41: 56, 42: 52, 43: 37, 44: 38, 45: 39, 46: 33, 47: 20, 48: 19, 49: 18, 50: 14, 51: 0, 52: 1, 53: 2, 54: 3, 55: 4, 56: 5, 57: 6, 58: 7, 59: 8, 60: 9, 61: 10, 62: 11, 63: 12, 64: 13, 65: 15, 66: 16, 67: 17, 68: 21, 69: 22, 70: 23, 71: 35, 72: 40, 73: 44, 74: 55, 75: 61, 76: 65, 77: 66, 78: 67, 79: 68, 80: 69, 81: 70, 82: 71, 83: 73, 84: 74, 85: 82, 86: 83, 87: 84, 88: 85, 89: 86, 90: 87, 91: 88, 92: 89, 93: 91, 94: 92, 95: 93, 96: 96, 97: 97, 98: 98, 99: 99, 100: 100, 101: 101, 102: 102, 103: 103, 104: 104, 105: 105, 106: 106, 107: 107, 108: 108, 109: 109, 110: 110, 111: 111, 112: 112, 113: 113, 114: 114, 115: 115, 116: 116, 117:

,name,num_qubits,num_gates,depth,depth_2q,num_swaps,transpile_time,ops,passmanager
0,cat_n65.qasm,65,130,66,64,0,0.059155,"{""measure"": 65, ""cx"": 64, ""h"": 1, ""barrier"": 1}",multi_sabre
1,dnn_n51.qasm,51,274,58,52,0,0.063979,"{""measure"": 51, ""ry"": 50, ""rz"": 50, ""cswap"": 2...",multi_sabre
2,ghz_n78.qasm,78,156,79,77,0,0.060152,"{""measure"": 78, ""cx"": 77, ""h"": 1, ""barrier"": 1}",multi_sabre
3,ising_n98.qasm,98,1170,16,4,0,0.071017,"{""rz"": 584, ""h"": 294, ""cx"": 194, ""measure"": 98...",multi_sabre
4,knn_67.qasm,67,102,36,0,0,0.017148,"{""ry"": 66, ""cswap"": 33, ""h"": 2, ""measure"": 1}",multi_sabre
5,multiplier_n45.qasm,45,698,462,234,0,0.013789,"{""ccx"": 378, ""cx"": 306, ""measure"": 9, ""x"": 5}",multi_sabre
6,swap_test_n83.qasm,83,126,44,0,0,0.007110,"{""rx"": 82, ""cswap"": 41, ""h"": 2, ""measure"": 1}",multi_sabre


In [14]:
# run the multi passmanger on all circuits
for qc_name, qc in circuits_dict.items():
    qc_tr = pm_multi_sabre.run(qc)

{0: 95, 1: 94, 2: 90, 3: 75, 4: 76, 5: 77, 6: 78, 7: 79, 8: 80, 9: 81, 10: 72, 11: 62, 12: 63, 13: 64, 14: 54, 15: 45, 16: 46, 17: 47, 18: 48, 19: 49, 20: 50, 21: 51, 22: 36, 23: 32, 24: 31, 25: 30, 26: 29, 27: 28, 28: 27, 29: 26, 30: 25, 31: 24, 32: 34, 33: 43, 34: 42, 35: 41, 36: 53, 37: 60, 38: 59, 39: 58, 40: 57, 41: 56, 42: 52, 43: 37, 44: 38, 45: 39, 46: 33, 47: 20, 48: 19, 49: 18, 50: 14, 51: 0, 52: 1, 53: 2, 54: 3, 55: 4, 56: 5, 57: 6, 58: 7, 59: 8, 60: 9, 61: 10, 62: 11, 63: 12, 64: 13, 65: 15, 66: 16, 67: 17, 68: 21, 69: 22, 70: 23, 71: 35, 72: 40, 73: 44, 74: 55, 75: 61, 76: 65, 77: 66, 78: 67, 79: 68, 80: 69, 81: 70, 82: 71, 83: 73, 84: 74, 85: 82, 86: 83, 87: 84, 88: 85, 89: 86, 90: 87, 91: 88, 92: 89, 93: 91, 94: 92, 95: 93, 96: 96, 97: 97, 98: 98, 99: 99, 100: 100, 101: 101, 102: 102, 103: 103, 104: 104, 105: 105, 106: 106, 107: 107, 108: 108, 109: 109, 110: 110, 111: 111, 112: 112, 113: 113, 114: 114, 115: 115, 116: 116, 117: 117, 118: 118, 119: 119, 120: 120, 121: 121,

In [15]:
layout_dict1 = {0: 95, 1: 94, 2: 90, 3: 75, 4: 76, 5: 77, 6: 78, 7: 79, 8: 80, 9: 81, 10: 72, 11: 62, 12: 63, 13: 64, 14: 54, 15: 45, 16: 46, 17: 47, 18: 48, 19: 49, 20: 50, 21: 51, 22: 36, 23: 32, 24: 31, 25: 30, 26: 29, 27: 28, 28: 27, 29: 26, 30: 25, 31: 24, 32: 34, 33: 43, 34: 42, 35: 41, 36: 53, 37: 60, 38: 59, 39: 58, 40: 57, 41: 56, 42: 52, 43: 37, 44: 38, 45: 39, 46: 33, 47: 20, 48: 19, 49: 18, 50: 14, 51: 0, 52: 1, 53: 2, 54: 3, 55: 4, 56: 5, 57: 6, 58: 7, 59: 8, 60: 9, 61: 10, 62: 11, 63: 12, 64: 13, 65: 15, 66: 16, 67: 17, 68: 21, 69: 22, 70: 23, 71: 35, 72: 40, 73: 44, 74: 55, 75: 61, 76: 65, 77: 66, 78: 67, 79: 68, 80: 69, 81: 70, 82: 71, 83: 73, 84: 74, 85: 82, 86: 83, 87: 84, 88: 85, 89: 86, 90: 87, 91: 88, 92: 89, 93: 91, 94: 92, 95: 93, 96: 96, 97: 97, 98: 98, 99: 99, 100: 100, 101: 101, 102: 102, 103: 103, 104: 104, 105: 105, 106: 106, 107: 107, 108: 108, 109: 109, 110: 110, 111: 111, 112: 112, 113: 113, 114: 114, 115: 115, 116: 116, 117: 117, 118: 118, 119: 119, 120: 120, 121: 121, 122: 122, 123: 123, 124: 124, 125: 125, 126: 126}
layout_dict2 = {0: 15, 1: 57, 2: 56, 3: 52, 4: 37, 5: 38, 6: 39, 7: 33, 8: 20, 9: 19, 10: 18, 11: 14, 12: 0, 13: 1, 14: 2, 15: 3, 16: 4, 17: 5, 18: 6, 19: 7, 20: 8, 21: 9, 22: 10, 23: 11, 24: 12, 25: 13, 26: 16, 27: 17, 28: 21, 29: 22, 30: 23, 31: 24, 32: 25, 33: 26, 34: 27, 35: 28, 36: 29, 37: 30, 38: 31, 39: 32, 40: 34, 41: 35, 42: 36, 43: 40, 44: 41, 45: 42, 46: 43, 47: 44, 48: 45, 49: 46, 50: 47, 51: 48, 52: 49, 53: 50, 54: 51, 55: 53, 56: 54, 57: 55, 58: 58, 59: 59, 60: 60, 61: 61, 62: 62, 63: 63, 64: 64, 65: 65, 66: 66, 67: 67, 68: 68, 69: 69, 70: 70, 71: 71, 72: 72, 73: 73, 74: 74, 75: 75, 76: 76, 77: 77, 78: 78, 79: 79, 80: 80, 81: 81, 82: 82, 83: 83, 84: 84, 85: 85, 86: 86, 87: 87, 88: 88, 89: 89, 90: 90, 91: 91, 92: 92, 93: 93, 94: 94, 95: 95, 96: 96, 97: 97, 98: 98, 99: 99, 100: 100, 101: 101, 102: 102, 103: 103, 104: 104, 105: 105, 106: 106, 107: 107, 108: 108, 109: 109, 110: 110, 111: 111, 112: 112, 113: 113, 114: 114, 115: 115, 116: 116, 117: 117, 118: 118, 119: 119, 120: 120, 121: 121, 122: 122, 123: 123, 124: 124, 125: 125, 126: 126}
layout_dict3 = {0: 66, 1: 73, 2: 85, 3: 84, 4: 83, 5: 92, 6: 102, 7: 101, 8: 100, 9: 99, 10: 98, 11: 97, 12: 96, 13: 95, 14: 94, 15: 90, 16: 75, 17: 76, 18: 77, 19: 78, 20: 79, 21: 80, 22: 81, 23: 72, 24: 62, 25: 63, 26: 64, 27: 54, 28: 45, 29: 46, 30: 47, 31: 48, 32: 49, 33: 50, 34: 51, 35: 36, 36: 32, 37: 31, 38: 30, 39: 29, 40: 28, 41: 27, 42: 26, 43: 25, 44: 24, 45: 34, 46: 43, 47: 42, 48: 41, 49: 53, 50: 60, 51: 59, 52: 58, 53: 57, 54: 56, 55: 52, 56: 37, 57: 38, 58: 39, 59: 33, 60: 20, 61: 19, 62: 18, 63: 14, 64: 0, 65: 1, 66: 2, 67: 3, 68: 4, 69: 5, 70: 6, 71: 7, 72: 8, 73: 9, 74: 10, 75: 11, 76: 12, 77: 13, 78: 15, 79: 16, 80: 17, 81: 21, 82: 22, 83: 23, 84: 35, 85: 40, 86: 44, 87: 55, 88: 61, 89: 65, 90: 67, 91: 68, 92: 69, 93: 70, 94: 71, 95: 74, 96: 82, 97: 86, 98: 87, 99: 88, 100: 89, 101: 91, 102: 93, 103: 103, 104: 104, 105: 105, 106: 106, 107: 107, 108: 108, 109: 109, 110: 110, 111: 111, 112: 112, 113: 113, 114: 114, 115: 115, 116: 116, 117: 117, 118: 118, 119: 119, 120: 120, 121: 121, 122: 122, 123: 123, 124: 124, 125: 125, 126: 126}
layout_dict4 = {0: 116, 1: 117, 2: 118, 3: 119, 4: 120, 5: 121, 6: 122, 7: 111, 8: 104, 9: 105, 10: 106, 11: 93, 12: 87, 13: 88, 14: 89, 15: 74, 16: 70, 17: 69, 18: 68, 19: 67, 20: 66, 21: 73, 22: 85, 23: 84, 24: 83, 25: 92, 26: 102, 27: 101, 28: 100, 29: 99, 30: 98, 31: 97, 32: 96, 33: 95, 34: 94, 35: 90, 36: 75, 37: 76, 38: 77, 39: 78, 40: 79, 41: 80, 42: 81, 43: 72, 44: 62, 45: 63, 46: 64, 47: 54, 48: 45, 49: 46, 50: 47, 51: 48, 52: 49, 53: 50, 54: 51, 55: 36, 56: 32, 57: 31, 58: 30, 59: 29, 60: 28, 61: 27, 62: 26, 63: 25, 64: 24, 65: 34, 66: 43, 67: 42, 68: 41, 69: 53, 70: 60, 71: 59, 72: 58, 73: 57, 74: 56, 75: 52, 76: 37, 77: 38, 78: 39, 79: 33, 80: 20, 81: 19, 82: 18, 83: 14, 84: 0, 85: 1, 86: 2, 87: 3, 88: 4, 89: 5, 90: 6, 91: 7, 92: 8, 93: 9, 94: 10, 95: 11, 96: 12, 97: 13, 98: 15, 99: 16, 100: 17, 101: 21, 102: 22, 103: 23, 104: 35, 105: 40, 106: 44, 107: 55, 108: 61, 109: 65, 110: 71, 111: 82, 112: 86, 113: 91, 114: 103, 115: 107, 116: 108, 117: 109, 118: 110, 119: 112, 120: 113, 121: 114, 122: 115, 123: 123, 124: 124, 125: 125, 126: 126}
layout_dict5 = {0: 50, 1: 49, 2: 80, 3: 118, 4: 113, 5: 74, 6: 29, 7: 48, 8: 78, 9: 107, 10: 124, 11: 45, 12: 105, 13: 33, 14: 38, 15: 7, 16: 9, 17: 56, 18: 106, 19: 123, 20: 85, 21: 121, 22: 43, 23: 54, 24: 126, 25: 83, 26: 81, 27: 47, 28: 68, 29: 62, 30: 30, 31: 99, 32: 65, 33: 20, 34: 67, 35: 102, 36: 119, 37: 117, 38: 36, 39: 51, 40: 14, 41: 84, 42: 82, 43: 98, 44: 26, 45: 21, 46: 12, 47: 73, 48: 55, 49: 116, 50: 15, 51: 86, 52: 17, 53: 88, 54: 23, 55: 104, 56: 52, 57: 16, 58: 58, 59: 72, 60: 94, 61: 63, 62: 19, 63: 87, 64: 53, 65: 75, 66: 115, 67: 42, 68: 41, 69: 57, 70: 59, 71: 91, 72: 44, 73: 108, 74: 18, 75: 101, 76: 114, 77: 22, 78: 64, 79: 1, 80: 31, 81: 8, 82: 39, 83: 61, 84: 2, 85: 10, 86: 112, 87: 28, 88: 97, 89: 46, 90: 13, 91: 125, 92: 92, 93: 89, 94: 24, 95: 66, 96: 32, 97: 69, 98: 6, 99: 100, 100: 109, 101: 90, 102: 70, 103: 120, 104: 111, 105: 71, 106: 35, 107: 25, 108: 4, 109: 0, 110: 3, 111: 95, 112: 77, 113: 34, 114: 103, 115: 40, 116: 37, 117: 79, 118: 5, 119: 122, 120: 93, 121: 11, 122: 96, 123: 76, 124: 27, 125: 110, 126: 60}
layout_dict6 = {0: 13, 1: 11, 2: 12, 3: 28, 4: 47, 5: 35, 6: 17, 7: 29, 8: 30, 9: 16, 10: 27, 11: 26, 12: 8, 13: 10, 14: 9, 15: 7, 16: 5, 17: 6, 18: 0, 19: 2, 20: 1, 21: 4, 22: 22, 23: 15, 24: 25, 25: 34, 26: 24, 27: 44, 28: 14, 29: 43, 30: 3, 31: 45, 32: 18, 33: 23, 34: 42, 35: 19, 36: 21, 37: 41, 38: 20, 39: 40, 40: 33, 41: 39, 42: 38, 43: 37, 44: 52, 45: 31, 46: 32, 47: 36, 48: 46, 49: 48, 50: 49, 51: 50, 52: 51, 53: 53, 54: 54, 55: 55, 56: 56, 57: 57, 58: 58, 59: 59, 60: 60, 61: 61, 62: 62, 63: 63, 64: 64, 65: 65, 66: 66, 67: 67, 68: 68, 69: 69, 70: 70, 71: 71, 72: 72, 73: 73, 74: 74, 75: 75, 76: 76, 77: 77, 78: 78, 79: 79, 80: 80, 81: 81, 82: 82, 83: 83, 84: 84, 85: 85, 86: 86, 87: 87, 88: 88, 89: 89, 90: 90, 91: 91, 92: 92, 93: 93, 94: 94, 95: 95, 96: 96, 97: 97, 98: 98, 99: 99, 100: 100, 101: 101, 102: 102, 103: 103, 104: 104, 105: 105, 106: 106, 107: 107, 108: 108, 109: 109, 110: 110, 111: 111, 112: 112, 113: 113, 114: 114, 115: 115, 116: 116, 117: 117, 118: 118, 119: 119, 120: 120, 121: 121, 122: 122, 123: 123, 124: 124, 125: 125, 126: 126}
layout_dict7 = {0: 50, 1: 49, 2: 80, 3: 118, 4: 113, 5: 74, 6: 29, 7: 48, 8: 78, 9: 107, 10: 124, 11: 45, 12: 105, 13: 33, 14: 38, 15: 7, 16: 9, 17: 56, 18: 106, 19: 123, 20: 85, 21: 121, 22: 43, 23: 54, 24: 126, 25: 83, 26: 81, 27: 47, 28: 68, 29: 62, 30: 30, 31: 99, 32: 65, 33: 20, 34: 67, 35: 102, 36: 119, 37: 117, 38: 36, 39: 51, 40: 14, 41: 84, 42: 82, 43: 98, 44: 26, 45: 21, 46: 12, 47: 73, 48: 55, 49: 116, 50: 15, 51: 86, 52: 17, 53: 88, 54: 23, 55: 104, 56: 52, 57: 16, 58: 58, 59: 72, 60: 94, 61: 63, 62: 19, 63: 87, 64: 53, 65: 75, 66: 115, 67: 42, 68: 41, 69: 57, 70: 59, 71: 91, 72: 44, 73: 108, 74: 18, 75: 101, 76: 114, 77: 22, 78: 64, 79: 1, 80: 31, 81: 8, 82: 39, 83: 61, 84: 2, 85: 10, 86: 112, 87: 28, 88: 97, 89: 46, 90: 13, 91: 125, 92: 92, 93: 89, 94: 24, 95: 66, 96: 32, 97: 69, 98: 6, 99: 100, 100: 109, 101: 90, 102: 70, 103: 120, 104: 111, 105: 71, 106: 35, 107: 25, 108: 4, 109: 0, 110: 3, 111: 95, 112: 77, 113: 34, 114: 103, 115: 40, 116: 37, 117: 79, 118: 5, 119: 122, 120: 93, 121: 11, 122: 96, 123: 76, 124: 27, 125: 110, 126: 60}

In [16]:
def convert_layout(layout_dict: Dict[int, int]) -> List[int]:
    return list(layout_dict.values())

layouts = [convert_layout(d) for d in [layout_dict1, layout_dict2, layout_dict3, layout_dict4, layout_dict5, layout_dict6, layout_dict7]]

qc = circuits_dict["cat_n65.qasm"]


pm = generate_preset_pass_manager(coupling_map=coupling_map, optimization_level=2, seed_transpiler=seed, initial_layout=layouts[0][:qc.num_qubits])
qc_tr = pm.run(qc)
print(f"Preset PassManager with initial layout: {count_swaps(qc_tr)} swaps, 2Q depth: {qc_tr.depth(lambda x: x.operation.num_qubits == 2)}")

Preset PassManager with initial layout: 0 swaps, 2Q depth: 64


In [17]:
# run the multi passmanger on all circuits
idx = 0
for qc_name, qc in circuits_dict.items():
    pm = generate_preset_pass_manager(coupling_map=coupling_map, optimization_level=2, seed_transpiler=seed, initial_layout=layouts[idx][:qc.num_qubits])
    qc_tr = pm.run(qc)
    print(f"Circuit {qc_name}, index {idx+1}: {count_swaps(qc_tr)} swaps, 2Q depth: {qc_tr.depth(lambda x: x.operation.num_qubits == 2)}")
    idx += 1
    

Circuit cat_n65.qasm, index 1: 0 swaps, 2Q depth: 64
Circuit dnn_n51.qasm, index 2: 0 swaps, 2Q depth: 52
Circuit ghz_n78.qasm, index 3: 0 swaps, 2Q depth: 77
Circuit ising_n98.qasm, index 4: 0 swaps, 2Q depth: 4
Circuit knn_67.qasm, index 5: 0 swaps, 2Q depth: 0
Circuit multiplier_n45.qasm, index 6: 0 swaps, 2Q depth: 234
Circuit swap_test_n83.qasm, index 7: 0 swaps, 2Q depth: 0


In [18]:
passes = []
def callback_func(**kwargs):
    t_pass = kwargs['pass_'].name()
    if t_pass == 'VF2Layout':
        passes.append(kwargs['pass_'])
    

In [19]:
# run the original preset passmanager on all circuits
for qc_name, qc in circuits_dict.items():
    qc_tr = pm_preset.run(qc,callback=callback_func)
    print(f"Circuit {qc_name} with original preset PM: {count_swaps(qc_tr)} swaps, 2Q depth: {qc_tr.depth(lambda x: x.operation.num_qubits == 2)}")

    stop_reason = passes[-1].property_set['VF2Layout_stop_reason']
    print(f"  - Stop reason: {stop_reason}\n")


Circuit cat_n65.qasm with original preset PM: 14 swaps, 2Q depth: 78
  - Stop reason: VF2LayoutStopReason.NO_SOLUTION_FOUND

Circuit dnn_n51.qasm with original preset PM: 6 swaps, 2Q depth: 65
  - Stop reason: VF2LayoutStopReason.NO_SOLUTION_FOUND

Circuit ghz_n78.qasm with original preset PM: 16 swaps, 2Q depth: 93
  - Stop reason: VF2LayoutStopReason.NO_SOLUTION_FOUND

Circuit ising_n98.qasm with original preset PM: 17 swaps, 2Q depth: 10
  - Stop reason: VF2LayoutStopReason.NO_SOLUTION_FOUND

Circuit knn_67.qasm with original preset PM: 0 swaps, 2Q depth: 0
  - Stop reason: VF2LayoutStopReason.NO_SOLUTION_FOUND

Circuit multiplier_n45.qasm with original preset PM: 0 swaps, 2Q depth: 234
  - Stop reason: VF2LayoutStopReason.NO_SOLUTION_FOUND

Circuit swap_test_n83.qasm with original preset PM: 0 swaps, 2Q depth: 0
  - Stop reason: VF2LayoutStopReason.NO_SOLUTION_FOUND

